In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Input
import os
import random as rn
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_model_optimization as tfmot
from tensorflow.keras import initializers
from tensorflow.keras.datasets import fashion_mnist, mnist

In [9]:
my_dataset = fashion_mnist
(train_images, train_labels), (test_images, test_labels) = my_dataset.load_data()
X = train_images.reshape(-1, 784)/255 
y = train_labels.reshape(-1, 1)
X_test = test_images.reshape(-1, 784)/255
y_test = test_labels.reshape(-1, 1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [10]:
def compile_model(model, lr=0.001):
    optim = optimizers.Adam(lr=lr)
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    
def get_model(initial_weights=None, compile_model_flag=True, lr=0.001, activation='relu', kernel_initializer=initializers.glorot_uniform()):
    reduce = 1
    input_dim=784
    output_size = 10
    # Creo el modelo
    
    inp = Input(shape=(input_dim,))
    first_layer_out = Dense(784*2//reduce, activation=activation, name='hidden_1', kernel_initializer=kernel_initializer, input_shape=(input_dim,))(inp)
    second_layer_out = Dense(784//reduce, activation=activation, name='hidden_2', kernel_initializer=kernel_initializer)(first_layer_out)
    output = Dense(output_size, name='Salida', kernel_initializer=kernel_initializer, activation='softmax')(second_layer_out)
    model = Model(inp, output)
    model_multiple_out = Model(inp, [first_layer_out, second_layer_out, output])
    first_layer_model = Model(inp, first_layer_out)
    second_layer_model = Model(inp, second_layer_out)
    if compile_model_flag:
        compile_model(model, lr=lr)
    if initial_weights is not None:
        model.save_weights(initial_weights)
    return model, model_multiple_out


In [31]:
X = np.random.normal(0., 0.1, (60000, 784))

In [32]:
model, model_multiple_out = get_model(activation='tanh', kernel_initializer=initializers.VarianceScaling(1, 'fan_avg'))
model_multiple_out.summary()

Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
hidden_1 (Dense)             (None, 1568)              1230880   
_________________________________________________________________
hidden_2 (Dense)             (None, 784)               1230096   
_________________________________________________________________
Salida (Dense)               (None, 10)                7850      
Total params: 2,468,826
Trainable params: 2,468,826
Non-trainable params: 0
_________________________________________________________________


In [33]:
out_first, out_second, out = model_multiple_out.predict(X)
print(out_first.std()/X.std(), out_second.std()/(out_first.std()))

0.811861680224519 1.1444752


In [34]:
X_norm = (X - X.mean(axis=0))/(X.std(axis=0)+1/255)
out_first, out_second, out = model_multiple_out.predict(X_norm)
print(out_first.std()/X.std(), out_second.std()/(out_first.std()))

5.559037874483199 0.8867515
